In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
import torch

In [ ]:
# Check for GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [ ]:
data = pd.read_csv('final_cybercrime_data.csv')

In [ ]:
class CybercrimeDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        super().__init__()
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)  # Properly return the length of the dataset as an integer

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )

        item = {key: val.squeeze(0) for key, val in encoding.items()}
        item['labels'] = torch.tensor(label, dtype=torch.long)
        return item


In [ ]:
# Encode labels
label_mapping = {label: idx for idx, label in enumerate(data['Legal Sections (BNS & IT Act 2000)'].unique())}
data['label'] = data['Legal Sections (BNS & IT Act 2000)'].map(label_mapping)

# Split data into train and test sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    data['Cybercrime Input'].tolist(), data['label'].tolist(), test_size=0.2, random_state=42
)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
train_dataset = CybercrimeDataset(train_texts, train_labels, tokenizer, max_length=128)
val_dataset = CybercrimeDataset(val_texts, val_labels, tokenizer, max_length=128)

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_mapping)).to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    learning_rate=2e-5,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    fp16=True,# Enables mixed precision training for faster computation on GPUs
    max_steps=10000
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

# Train the model
trainer.train()

# Save the model
trainer.save_model("fine_tuned_bert_cybercrime")

<ipython-input-11-7217c208c983>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss
1,3.308800,3.168407
2,1.984900,1.745792
3,0.683300,0.495983
4,0.124400,0.087732


KeyboardInterrupt: 

In [ ]:
# Predict function with GPU
def predict(texts):
    model.eval()
    encodings = tokenizer(
        texts,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_tensors="pt"
    ).to(device)  # Move data to GPU
    with torch.no_grad():
        outputs = model(**encodings)
    predictions = torch.argmax(outputs.logits, dim=1).tolist()
    return [list(label_mapping.keys())[list(label_mapping.values()).index(p)] for p in predictions]

# Interactive prediction
def user_input_prediction():
    while True:
        user_input = input("Enter text for prediction (or type 'exit' to quit): ").strip()
        if user_input.lower() == 'exit':
            print("Exiting...")
            break
        predicted_label = predict([user_input])[0]
        print(f"Predicted Label: {predicted_label}\n")

# Evaluate the model with GPU
def evaluate_model():
    val_texts_list = val_texts
    val_labels_list = [list(label_mapping.keys())[list(label_mapping.values()).index(label)] for label in val_labels]
    predictions = predict(val_texts_list)

    correct = sum([pred == true for pred, true in zip(predictions, val_labels_list)])
    accuracy = correct / len(val_labels_list)
    print(f"Validation Accuracy: {accuracy:.2f}")

# Example usage
if __name__ == "__main__":
    # User input predictions
    print("Interactive Prediction Mode:")
    user_input_prediction()

    # Evaluate model
    print("Evaluating the model...")
    evaluate_model()


Interactive Prediction Mode:
Enter text for prediction (or type 'exit' to quit): my website got hacked
Predicted Label: BNS: Section 330, 336, IT Act: Section 43, 70

Enter text for prediction (or type 'exit' to quit): i love you
Predicted Label: BNS: Section 333, IT Act: Section 67

Enter text for prediction (or type 'exit' to quit): exit
Exiting...
Evaluating the model...
Validation Accuracy: 1.00
